In [ ]:
import gc, os, random, pickle
import pandas as pd
import numpy as np
import scipy
from scipy import stats

# 导入模型和sklearn
import xgboost as xgb
import lightgbm as lgb
import catboost as cat
from lightgbm import LGBMClassifier, LGBMRegressor
from xgboost import XGBClassifier, XGBRegressor
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.metrics import roc_auc_score, accuracy_score,auc, log_loss
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from tqdm import tqdm 
from imblearn.over_sampling import SMOTE
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',100)
    
'''seaborn and matplotlib'''
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus']=False
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

'''plotly'''
import plotly.offline as py
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go

'''按markdown格式展现数据'''
init_notebook_mode(connected = True)
from IPython.display import Markdown

def bold(string):
    display(Markdown(string))


In [ ]:
# 加载数据
train = pd.read_csv("../data/train.csv")
train_label = pd.read_csv("../data/train_label.csv")
test = pd.read_csv("../data/test.csv")
print(train.shape, train_label.shape, test.shape)



In [ ]:
print(train_label['Label'].value_counts()) # 正负样本极度不均衡 需要采样

#采样前， 先对不相关的特征做剔除，填充空值

In [ ]:
# 特征工程

train.dtypes

## 将特征可以分为两大类

```
第1大类：
ID                   int64     ==> 删除
行业代码               float64  ==> 删除
邮政编码                object  ==> 删除 没啥用

企业类型               float64  ==> 类别
登记机关               float64  ==> 类别
企业状态               float64  ==> 类别
行业门类               float64  ==> 类别
企业类别               float64  ==> 类别
管辖机关               float64  ==> 类别

经营期限自               object ==> 时间特征 ==> 时间类特征格式有问题，估计要转换，暂时不知道怎么用，先删除
经营期限至               object ==> 时间特征
核准日期                object  ==> 时间特征
成立日期                object  ==> 时间特征
注销时间                object  ==> 时间特征

经营范围                object  ==>  列表， 需要转为列表范围,

投资总额               float64  ==> 数值 极度稀疏，1.4w数据中约1.3w缺失
注册资本               float64  ==> 数值
增值税                 float64  ==> 数值
企业所得税             float64   ==> 数值
印花税                 float64   ==> 数值
教育费                 float64   ==> 数值
城建税                 float64   ==> 数值

时间特征 time_feature = ['经营期限自','经营期限至', '核准日期', '成立日期', '注销时间']
需要删除列 drop_feature = ['ID', '邮政编码', '行业代码', '经营范围']
类别特征   categoty_feature = ['企业类型', '登记机关', '企业状态', '行业门类','企业类别', '管辖机关']
数值型特征  ['投资总额', '注册资本', '增值税', '企业所得税', '印花税', '教育费', '城建税']  
需要构建的特征：==> baseline下看到的
'企业所得税比增值税'， 也可以类似构建 '企业所得税'和其他几个数值型特征之比


第2大类：所有包含年末 年初的数据-- 除开第1大类的其他大类
递延收益_年初数           float64
长期负债合计_年初数         float64
长期借款_年初数           float64
长期应付款_年初数          float64
短期借款_年初数           float64
递延所得税负债_年初数        float64
非流动负债合计_年初数        float64
负债合计_年初数           float64
负债和所有者权益总计_年初数     float64
交易性金融负债_年初数        float64
流动负债合计_年初数         float64
其他非流动负债_年初数        float64
其他负债（或长期负债）_年初数    float64
其他流动负债_年初数         float64
其他应付款_年初数          float64
其他应交款_年初数          float64
实收资本（股本）_年初数       float64
实收资本（或股本）净额_年初数    float64
所有者权益合计_年初数        float64
未分配利润_年初数          float64
小企业_应付利润_年初数       float64
应付福利费_年初数          float64
应付股利_年初数           float64
应付利息_年初数           float64
应付票据_年初数           float64
应付职工薪酬_年初数         float64
应付债券_年初数           float64
应付账款_年初数           float64
预计负债_年初数           float64
应交税费_年初数           float64
一年内到期的非流动负债_年初数    float64
预提费用_年初数           float64
预收款项_年初数           float64
盈余公积_年初数           float64
资本公积_年初数           float64
专项应付款_年初数          float64
递延收益_年末数           float64
长期负债合计_年末数         float64
长期借款_年末数           float64
长期应付款_年末数          float64
短期借款_年末数           float64
递延所得税负债_年末数        float64
非流动负债合计_年末数        float64
负债合计_年末数           float64
负债和所有者权益总计_年末数     float64
交易性金融负债_年末数        float64
流动负债合计_年末数         float64
其他非流动负债_年末数        float64
其他负债（或长期负债）_年末数    float64
其他流动负债_年末数         float64
其他应付款_年末数          float64
其他应交款_年末数          float64
实收资本（股本）_年末数       float64
实收资本（或股本）净额_年末数    float64
所有者权益合计_年末数        float64
未分配利润_年末数          float64
小企业_应付利润_年末数       float64
应付福利费_年末数          float64
应付股利_年末数           float64
应付利息_年末数           float64
应付票据_年末数           float64
应付职工薪酬_年末数         float64
应付债券_年末数           float64
应付账款_年末数           float64
预计负债_年末数           float64
应交税费_年末数           float64
一年内到期的非流动负债_年末数    float64
预提费用_年末数           float64
预收款项_年末数           float64
盈余公积_年末数           float64
资本公积_年末数           float64
专项应付款_年末数          float64
货币资金_年初数           float64
存货_年初数             float64
存货中的原材料_年初数        float64
存货中的周转材料_年初数       float64
长期待摊费用_年初数         float64
长期股权投资_年初数         float64
长期投资合计_年初数         float64
长期应收款_年初数          float64
持有至到期投资_年初数        float64
短期投资_年初数           float64
待摊费用_年初数           float64
递延所得税资产_年初数        float64
非流动资产合计_年初数        float64
工程物资_年初数           float64
固定资产合计_年初数         float64
固定资产净额_年初数         float64
固定资产净值_年初数         float64
固定资产清理_年初数         float64
固定资产原价_年初数         float64
交易性金融资产_年初数        float64
库存商品(产成品)_年初数      float64
开发支出_年初数           float64
可供出售金融资产_年初数       float64
流动资产合计_年初数         float64
其他非流动资产_年初数        float64
其他流动资产_年初数         float64
其他应收款_年初数          float64
生产性生物资产_年初数        float64
商誉_年初数             float64
投资性房地产_年初数         float64
无形资产_年初数           float64
无形资产及其他资产合计_年初数    float64
应收账款_年初数           float64
一年内到期的非流动资产_年初数    float64
油气资产_年初数           float64
应收补贴款_年初数          float64
应收股利_年初数           float64
应收利息_年初数           float64
应收票据_年初数           float64
预付款项_年初数           float64
资产总计_年初数           float64
在建工程_年初数           float64
货币资金_年末数           float64
存货_年末数             float64
存货中的原材料_年末数        float64
存货中的周转材料_年末数       float64
长期待摊费用_年末数         float64
长期股权投资_年末数         float64
长期投资合计_年末数         float64
长期应收款_年末数          float64
持有至到期投资_年末数        float64
短期投资_年末数           float64
待摊费用_年末数           float64
递延所得税资产_年末数        float64
非流动资产合计_年末数        float64
工程物资_年末数           float64
固定资产合计_年末数         float64
固定资产净额_年末数         float64
固定资产净值_年末数         float64
固定资产清理_年末数         float64
固定资产原价_年末数         float64
交易性金融资产_年末数        float64
库存商品(产成品)_年末数      float64
开发支出_年末数           float64
可供出售金融资产_年末数       float64
流动资产合计_年末数         float64
其他非流动资产_年末数        float64
其他流动资产_年末数         float64
其他应收款_年末数          float64
生产性生物资产_年末数        float64
商誉_年末数             float64
投资性房地产_年末数         float64
无形资产_年末数           float64
无形资产及其他资产合计_年末数    float64
应收账款_年末数           float64
一年内到期的非流动资产_年末数    float64
油气资产_年末数           float64
应收补贴款_年末数          float64
应收股利_年末数           float64
应收利息_年末数           float64
应收票据_年末数           float64
预付款项_年末数           float64
资产总计_年末数           float64
在建工程_年末数           float64
```

In [ ]:
time_feature = ['经营期限自','经营期限至', '核准日期', '成立日期', '注销时间']
drop_feature = ['ID', '邮政编码', '行业代码', '经营范围']
categoty_feature = ['企业类型', '登记机关', '企业状态', '行业门类','企业类别', '管辖机关']
tax_feature= ['投资总额', '注册资本', '增值税', '企业所得税', '印花税', '教育费', '城建税']  # 税收类特征
#需要构建的特征：==> baseline下看到的
# '企业所得税比增值税'， 也可以类似构建 '企业所得税'和其他几个数值型特征之比


In [ ]:
train['经营范围2'] =train["经营范围"].map(lambda x : len(x))
test['经营范围2'] =test["经营范围"].map(lambda x : len(x))


In [ ]:
drop_feature += time_feature
drop_feature += ['经营范围']
print(drop_feature)

In [ ]:
train.drop(columns=drop_feature, inplace=True)
test.drop(columns=drop_feature, inplace=True)
print(train.shape, test.shape)

In [ ]:
## smote 过采样，将训练集样本均衡
print(train.shape, train_label.shape)
print(train_label['Label'].value_counts())

In [ ]:

train_head = train.iloc[0:0]
train_label_head = train_label.iloc[0:0]
display(train_head.head())
display(train_label_head.head())

In [ ]:

train_smo = train.iloc[1:]
train_label_smo = train_label.iloc[1:]
train_smo.head(2)

In [ ]:
from imblearn import over_sampling
from imblearn.over_sampling import SMOTE

train.fillna(0, inplace=True)
smo = SMOTE(random_state=42, )
train_smo, y_smo = smo.fit_sample(train, train_label['Label'])

In [ ]:
train_smo = pd.DataFrame(train_smo)
y_smo = pd.DataFrame(y_smo)
print(train_smo.shape, y_smo.shape)



In [ ]:
display(train_smo.head(2))
display(y_smo.head(2))
print(train_smo.shape, y_smo.shape, test.shape)

In [ ]:
train['企业所得税与增值税之比'] = train['企业所得税']/ train['增值税']
test['企业所得税与增值税之比'] = test['企业所得税']/test['增值税']


In [ ]:
#第1类特征的列名
# categoty_feature = ['企业类型', '登记机关', '企业状态', '行业门类','企业类别', '管辖机关']
# tax_feature= ['投资总额', '注册资本', '增值税', '企业所得税', '印花税', '教育费', '城建税']  # 税收类特征

feature_class1 = categoty_feature +  tax_feature + ['经营范围2', '企业所得税与增值税之比']
print(feature_class1)

In [ ]:
# 处理第2类特征

In [ ]:
train_smo['递延收益'] = train_smo['递延收益_年末数'] - train_smo['递延收益_年初数']
train_smo['长期负债合计'] = train_smo['长期负债合计_年末数'] - train_smo['长期负债合计_年初数']
train_smo['长期借款'] = train_smo['长期借款_年末数'] - train_smo['长期借款_年初数']
train_smo['长期应付款'] = train_smo['长期应付款_年末数'] - train_smo['长期应付款_年初数']
train_smo['短期借款'] = train_smo['短期借款_年末数'] - train_smo['短期借款_年初数']
train_smo['递延所得税负债'] = train_smo['递延所得税负债_年末数'] - train_smo['递延所得税负债_年初数']
train_smo['非流动负债合计'] = train_smo['非流动负债合计_年末数'] - train_smo['非流动负债合计_年初数']
train_smo['负债合计'] = train_smo['负债合计_年末数'] - train_smo['负债合计_年初数']
train_smo['负债和所有者权益总计'] = train_smo['负债和所有者权益总计_年末数'] - train_smo['负债和所有者权益总计_年初数']
train_smo['交易性金融负债'] = train_smo['交易性金融负债_年末数'] - train_smo['交易性金融负债_年初数']
train_smo['流动负债合计'] = train_smo['流动负债合计_年末数'] - train_smo['流动负债合计_年初数']
train_smo['其他非流动负债'] = train_smo['其他非流动负债_年末数'] - train_smo['其他非流动负债_年初数']
train_smo['其他负债（或长期负债）'] = train_smo['其他负债（或长期负债）_年末数'] - train_smo['其他负债（或长期负债）_年初数']
train_smo['其他流动负债'] = train_smo['其他流动负债_年末数'] - train_smo['其他流动负债_年初数']
train_smo['其他应付款'] = train_smo['其他应付款_年末数'] - train_smo['其他应付款_年初数']
train_smo['其他应交款'] = train_smo['其他应交款_年末数'] - train_smo['其他应交款_年初数']
train_smo['实收资本（股本）'] = train_smo['实收资本（股本）_年末数'] - train_smo['实收资本（股本）_年初数']
train_smo['实收资本（或股本）净额'] = train_smo['实收资本（或股本）净额_年末数'] - train_smo['实收资本（或股本）净额_年初数']
train_smo['所有者权益合计'] = train_smo['所有者权益合计_年末数'] - train_smo['所有者权益合计_年初数']
train_smo['未分配利润'] = train_smo['未分配利润_年末数'] - train_smo['未分配利润_年初数']
train_smo['小企业_应付利润'] = train_smo['小企业_应付利润_年末数'] - train_smo['小企业_应付利润_年初数']
train_smo['应付福利费'] = train_smo['应付福利费_年末数'] - train_smo['应付福利费_年初数']
train_smo['应付股利'] = train_smo['应付股利_年末数'] - train_smo['应付股利_年初数']
train_smo['应付利息'] = train_smo['应付利息_年末数'] - train_smo['应付利息_年初数']
train_smo['应付票据'] = train_smo['应付票据_年末数'] - train_smo['应付票据_年初数']
train_smo['应付职工薪酬'] = train_smo['应付职工薪酬_年末数'] - train_smo['应付职工薪酬_年初数']
train_smo['应付债券'] = train_smo['应付债券_年末数'] - train_smo['应付债券_年初数']
train_smo['应付账款'] = train_smo['应付账款_年末数'] - train_smo['应付账款_年初数']
train_smo['预计负债'] = train_smo['预计负债_年末数'] - train_smo['预计负债_年初数']
train_smo['应交税费'] = train_smo['应交税费_年末数'] - train_smo['应交税费_年初数']
train_smo['一年内到期的非流动负债'] = train_smo['一年内到期的非流动负债_年末数'] - train_smo['一年内到期的非流动负债_年初数']
train_smo['预提费用'] = train_smo['预提费用_年末数'] - train_smo['预提费用_年初数']
train_smo['预收款项'] = train_smo['预收款项_年末数'] - train_smo['预收款项_年初数']
train_smo['盈余公积'] = train_smo['盈余公积_年末数'] - train_smo['盈余公积_年初数']
train_smo['资本公积'] = train_smo['资本公积_年末数'] - train_smo['资本公积_年初数']
train_smo['专项应付款'] = train_smo['专项应付款_年末数'] - train_smo['专项应付款_年初数']
train_smo['货币资金'] = train_smo['货币资金_年末数'] - train_smo['货币资金_年初数']
train_smo['存货'] = train_smo['存货_年末数'] - train_smo['存货_年初数']
train_smo['存货中的原材料'] = train_smo['存货中的原材料_年末数'] - train_smo['存货中的原材料_年初数']
train_smo['存货中的周转材料'] = train_smo['存货中的周转材料_年末数'] - train_smo['存货中的周转材料_年初数']
train_smo['长期待摊费用'] = train_smo['长期待摊费用_年末数'] - train_smo['长期待摊费用_年初数']
train_smo['长期股权投资'] = train_smo['长期股权投资_年末数'] - train_smo['长期股权投资_年初数']
train_smo['长期投资合计'] = train_smo['长期投资合计_年末数'] - train_smo['长期投资合计_年初数']
train_smo['长期应收款'] = train_smo['长期应收款_年末数'] - train_smo['长期应收款_年初数']
train_smo['持有至到期投资'] = train_smo['持有至到期投资_年末数'] - train_smo['持有至到期投资_年初数']
train_smo['短期投资'] = train_smo['短期投资_年末数'] - train_smo['短期投资_年初数']
train_smo['待摊费用'] = train_smo['待摊费用_年末数'] - train_smo['待摊费用_年初数']
train_smo['递延所得税资产'] = train_smo['递延所得税资产_年末数'] - train_smo['递延所得税资产_年初数']
train_smo['非流动资产合计'] = train_smo['非流动资产合计_年末数'] - train_smo['非流动资产合计_年初数']
train_smo['工程物资'] = train_smo['工程物资_年末数'] - train_smo['工程物资_年初数']
train_smo['固定资产合计'] = train_smo['固定资产合计_年末数'] - train_smo['固定资产合计_年初数']
train_smo['固定资产净额'] = train_smo['固定资产净额_年末数'] - train_smo['固定资产净额_年初数']
train_smo['固定资产净值'] = train_smo['固定资产净值_年末数'] - train_smo['固定资产净值_年初数']
train_smo['固定资产清理'] = train_smo['固定资产清理_年末数'] - train_smo['固定资产清理_年初数']
train_smo['固定资产原价'] = train_smo['固定资产原价_年末数'] - train_smo['固定资产原价_年初数']
train_smo['交易性金融资产'] = train_smo['交易性金融资产_年末数'] - train_smo['交易性金融资产_年初数']
train_smo['库存商品(产成品)'] = train_smo['库存商品(产成品)_年末数'] - train_smo['库存商品(产成品)_年初数']
train_smo['开发支出'] = train_smo['开发支出_年末数'] - train_smo['开发支出_年初数']
train_smo['可供出售金融资产'] = train_smo['可供出售金融资产_年末数'] - train_smo['可供出售金融资产_年初数']
train_smo['流动资产合计'] = train_smo['流动资产合计_年末数'] - train_smo['流动资产合计_年初数']
train_smo['其他非流动资产'] = train_smo['其他非流动资产_年末数'] - train_smo['其他非流动资产_年初数']
train_smo['其他流动资产'] = train_smo['其他流动资产_年末数'] - train_smo['其他流动资产_年初数']
train_smo['其他应收款'] = train_smo['其他应收款_年末数'] - train_smo['其他应收款_年初数']
train_smo['生产性生物资产'] = train_smo['生产性生物资产_年末数'] - train_smo['生产性生物资产_年初数']
train_smo['商誉'] = train_smo['商誉_年末数'] - train_smo['商誉_年初数']
train_smo['投资性房地产'] = train_smo['投资性房地产_年末数'] - train_smo['投资性房地产_年初数']
train_smo['无形资产'] = train_smo['无形资产_年末数'] - train_smo['无形资产_年初数']
train_smo['无形资产及其他资产合计'] = train_smo['无形资产及其他资产合计_年末数'] - train_smo['无形资产及其他资产合计_年初数']
train_smo['应收账款'] = train_smo['应收账款_年末数'] - train_smo['应收账款_年初数']
train_smo['一年内到期的非流动资产'] = train_smo['一年内到期的非流动资产_年末数'] - train_smo['一年内到期的非流动资产_年初数']
train_smo['油气资产'] = train_smo['油气资产_年末数'] - train_smo['油气资产_年初数']
train_smo['应收补贴款'] = train_smo['应收补贴款_年末数'] - train_smo['应收补贴款_年初数']
train_smo['应收股利'] = train_smo['应收股利_年末数'] - train_smo['应收股利_年初数']
train_smo['应收利息'] = train_smo['应收利息_年末数'] - train_smo['应收利息_年初数']
train_smo['应收票据'] = train_smo['应收票据_年末数'] - train_smo['应收票据_年初数']
train_smo['预付款项'] = train_smo['预付款项_年末数'] - train_smo['预付款项_年初数']
train_smo['资产总计'] = train_smo['资产总计_年末数'] - train_smo['资产总计_年初数']
train_smo['在建工程'] = train_smo['在建工程_年末数'] - train_smo['在建工程_年初数']


feature_class2  = ['递延收益', '长期负债合计', '长期借款', '长期应付款', '短期借款', '递延所得税负债', '非流动负债合计', '负债合计', '负债和所有者权益总计', '交易性金融负债', '流动负债合计', '其他非流动负债', '其他负债（或长期负债）', '其他流动负债', '其他应付款', '其他应交款', '实收资本（股本）', '实收资本（或股本）净额', '所有者权益合计', '未分配利润', '小企业_应付利润', '应付福利费', '应付股利', '应付利息', '应付票据', '应付职工薪酬', '应付债券', '应付账款', '预计负债', '应交税费', '一年内到期的非流动负债', '预提费用', '预收款项', '盈余公积', '资本公积', '专项应付款', '货币资金', '存货', '存货中的原材料', '存货中的周转材料', '长期待摊费用', '长期股权投资', '长期投资合计', '长期应收款', '持有至到期投资', '短期投资', '待摊费用', '递延所得税资产', '非流动资产合计', '工程物资', '固定资产合计', '固定资产净额', '固定资产净值', '固定资产清理', '固定资产原价', '交易性金融资产', '库存商品(产成品)', '开发支出', '可供出售金融资产', '流动资产合计', '其他非流动资产', '其他流动资产', '其他应收款', '生产性生物资产', '商誉', '投资性房地产', '无形资产', '无形资产及其他资产合计', '应收账款', '一年内到期的非流动资产', '油气资产', '应收补贴款', '应收股利', '应收利息', '应收票据', '预付款项', '资产总计', '在建工程']


In [ ]:
test['递延收益'] = test['递延收益_年末数'] - test['递延收益_年初数']
test['长期负债合计'] = test['长期负债合计_年末数'] - test['长期负债合计_年初数']
test['长期借款'] = test['长期借款_年末数'] - test['长期借款_年初数']
test['长期应付款'] = test['长期应付款_年末数'] - test['长期应付款_年初数']
test['短期借款'] = test['短期借款_年末数'] - test['短期借款_年初数']
test['递延所得税负债'] = test['递延所得税负债_年末数'] - test['递延所得税负债_年初数']
test['非流动负债合计'] = test['非流动负债合计_年末数'] - test['非流动负债合计_年初数']
test['负债合计'] = test['负债合计_年末数'] - test['负债合计_年初数']
test['负债和所有者权益总计'] = test['负债和所有者权益总计_年末数'] - test['负债和所有者权益总计_年初数']
test['交易性金融负债'] = test['交易性金融负债_年末数'] - test['交易性金融负债_年初数']
test['流动负债合计'] = test['流动负债合计_年末数'] - test['流动负债合计_年初数']
test['其他非流动负债'] = test['其他非流动负债_年末数'] - test['其他非流动负债_年初数']
test['其他负债（或长期负债）'] = test['其他负债（或长期负债）_年末数'] - test['其他负债（或长期负债）_年初数']
test['其他流动负债'] = test['其他流动负债_年末数'] - test['其他流动负债_年初数']
test['其他应付款'] = test['其他应付款_年末数'] - test['其他应付款_年初数']
test['其他应交款'] = test['其他应交款_年末数'] - test['其他应交款_年初数']
test['实收资本（股本）'] = test['实收资本（股本）_年末数'] - test['实收资本（股本）_年初数']
test['实收资本（或股本）净额'] = test['实收资本（或股本）净额_年末数'] - test['实收资本（或股本）净额_年初数']
test['所有者权益合计'] = test['所有者权益合计_年末数'] - test['所有者权益合计_年初数']
test['未分配利润'] = test['未分配利润_年末数'] - test['未分配利润_年初数']
test['小企业_应付利润'] = test['小企业_应付利润_年末数'] - test['小企业_应付利润_年初数']
test['应付福利费'] = test['应付福利费_年末数'] - test['应付福利费_年初数']
test['应付股利'] = test['应付股利_年末数'] - test['应付股利_年初数']
test['应付利息'] = test['应付利息_年末数'] - test['应付利息_年初数']
test['应付票据'] = test['应付票据_年末数'] - test['应付票据_年初数']
test['应付职工薪酬'] = test['应付职工薪酬_年末数'] - test['应付职工薪酬_年初数']
test['应付债券'] = test['应付债券_年末数'] - test['应付债券_年初数']
test['应付账款'] = test['应付账款_年末数'] - test['应付账款_年初数']
test['预计负债'] = test['预计负债_年末数'] - test['预计负债_年初数']
test['应交税费'] = test['应交税费_年末数'] - test['应交税费_年初数']
test['一年内到期的非流动负债'] = test['一年内到期的非流动负债_年末数'] - test['一年内到期的非流动负债_年初数']
test['预提费用'] = test['预提费用_年末数'] - test['预提费用_年初数']
test['预收款项'] = test['预收款项_年末数'] - test['预收款项_年初数']
test['盈余公积'] = test['盈余公积_年末数'] - test['盈余公积_年初数']
test['资本公积'] = test['资本公积_年末数'] - test['资本公积_年初数']
test['专项应付款'] = test['专项应付款_年末数'] - test['专项应付款_年初数']
test['货币资金'] = test['货币资金_年末数'] - test['货币资金_年初数']
test['存货'] = test['存货_年末数'] - test['存货_年初数']
test['存货中的原材料'] = test['存货中的原材料_年末数'] - test['存货中的原材料_年初数']
test['存货中的周转材料'] = test['存货中的周转材料_年末数'] - test['存货中的周转材料_年初数']
test['长期待摊费用'] = test['长期待摊费用_年末数'] - test['长期待摊费用_年初数']
test['长期股权投资'] = test['长期股权投资_年末数'] - test['长期股权投资_年初数']
test['长期投资合计'] = test['长期投资合计_年末数'] - test['长期投资合计_年初数']
test['长期应收款'] = test['长期应收款_年末数'] - test['长期应收款_年初数']
test['持有至到期投资'] = test['持有至到期投资_年末数'] - test['持有至到期投资_年初数']
test['短期投资'] = test['短期投资_年末数'] - test['短期投资_年初数']
test['待摊费用'] = test['待摊费用_年末数'] - test['待摊费用_年初数']
test['递延所得税资产'] = test['递延所得税资产_年末数'] - test['递延所得税资产_年初数']
test['非流动资产合计'] = test['非流动资产合计_年末数'] - test['非流动资产合计_年初数']
test['工程物资'] = test['工程物资_年末数'] - test['工程物资_年初数']
test['固定资产合计'] = test['固定资产合计_年末数'] - test['固定资产合计_年初数']
test['固定资产净额'] = test['固定资产净额_年末数'] - test['固定资产净额_年初数']
test['固定资产净值'] = test['固定资产净值_年末数'] - test['固定资产净值_年初数']
test['固定资产清理'] = test['固定资产清理_年末数'] - test['固定资产清理_年初数']
test['固定资产原价'] = test['固定资产原价_年末数'] - test['固定资产原价_年初数']
test['交易性金融资产'] = test['交易性金融资产_年末数'] - test['交易性金融资产_年初数']
test['库存商品(产成品)'] = test['库存商品(产成品)_年末数'] - test['库存商品(产成品)_年初数']
test['开发支出'] = test['开发支出_年末数'] - test['开发支出_年初数']
test['可供出售金融资产'] = test['可供出售金融资产_年末数'] - test['可供出售金融资产_年初数']
test['流动资产合计'] = test['流动资产合计_年末数'] - test['流动资产合计_年初数']
test['其他非流动资产'] = test['其他非流动资产_年末数'] - test['其他非流动资产_年初数']
test['其他流动资产'] = test['其他流动资产_年末数'] - test['其他流动资产_年初数']
test['其他应收款'] = test['其他应收款_年末数'] - test['其他应收款_年初数']
test['生产性生物资产'] = test['生产性生物资产_年末数'] - test['生产性生物资产_年初数']
test['商誉'] = test['商誉_年末数'] - test['商誉_年初数']
test['投资性房地产'] = test['投资性房地产_年末数'] - test['投资性房地产_年初数']
test['无形资产'] = test['无形资产_年末数'] - test['无形资产_年初数']
test['无形资产及其他资产合计'] = test['无形资产及其他资产合计_年末数'] - test['无形资产及其他资产合计_年初数']
test['应收账款'] = test['应收账款_年末数'] - test['应收账款_年初数']
test['一年内到期的非流动资产'] = test['一年内到期的非流动资产_年末数'] - test['一年内到期的非流动资产_年初数']
test['油气资产'] = test['油气资产_年末数'] - test['油气资产_年初数']
test['应收补贴款'] = test['应收补贴款_年末数'] - test['应收补贴款_年初数']
test['应收股利'] = test['应收股利_年末数'] - test['应收股利_年初数']
test['应收利息'] = test['应收利息_年末数'] - test['应收利息_年初数']
test['应收票据'] = test['应收票据_年末数'] - test['应收票据_年初数']
test['预付款项'] = test['预付款项_年末数'] - test['预付款项_年初数']
test['资产总计'] = test['资产总计_年末数'] - test['资产总计_年初数']
test['在建工程'] = test['在建工程_年末数'] - test['在建工程_年初数']

In [ ]:
train_smo['企业所得税与增值税之比'] = train_smo['企业所得税']/ train_smo['增值税']
test['企业所得税与增值税之比'] = test['企业所得税']/test['增值税']

In [ ]:
# 训练模型

In [ ]:

fi = []
cv_score = []
test_pred = np.zeros((test.shape[0],))
skf = StratifiedKFold(n_splits=10, random_state=2019, shuffle=True)



for index, (train_index, test_index) in enumerate(skf.split(train_smo, y_smo)):
    print(index)
    train_x, test_x, train_y, test_y = train_smo.iloc[train_index],train_smo.iloc[test_index], y_smo.iloc[train_index], y_smo.iloc[test_index]
    cat_model = CatBoostRegressor(iterations=300,
                              depth=5,
                              learning_rate=0.1, 
                              loss_function='RMSE', 
                              verbose=20, 
#                               early_stopping_rounds=20
                             )
    cat_model.fit(train_x, train_y, 
             eval_set=(test_x, test_y))
    y_val = cat_model.predict(test_x)
    print("roc_auc:", roc_auc_score(test_y, y_val))
    cv_score.append(roc_auc_score(test_y,y_val))
    print("cv_score:", cv_score[index])
    test_pred += cat_model.predict(test) / 10
print(cv_score)
test_pred = [0.0 if r < 0  else r for r in test_pred]
test_pred = [1.0 if r > 1.0  else r for r in test_pred]
submission = pd.read_csv('../data/submission.csv')
submission['Label'] = test_pred
submission.to_csv('../result/catboost_1207.csv',index=False)

# 过拟合

In [ ]:
feature_name_uesd = train_smo.columns
from matplotlib import pyplot
bold("**catboost特征重要性**")
print(cat_model.feature_importances_)
pyplot.bar(range(len(cat_model.feature_importances_)), cat_model.feature_importances_)
feature_important = dict(zip(feature_name_uesd, cat_model.feature_importances_))
print("catboost特征重要性:", feature_important)
feature_import_usefull = [key for key, value in feature_important.items() if value >0.0]
print("\n\n catboost 重要性 > 0 特征类别：", len(feature_import_usefull),  feature_import_usefull)


In [ ]:
feature_selected = ['企业类型', '登记机关', '企业状态', '投资总额', '注册资本', '行业门类', '企业类别', '管辖机关', '增值税', '企业所得税', '印花税', '教育费', '城建税', '长期借款_年初数', '短期借款_年初数', '非流动负债合计_年初数', '负债合计_年初数', '负债和所有者权益总计_年初数', '流动负债合计_年初数', '其他流动负债_年初数', '其他应付款_年初数', '实收资本（股本）_年初数', '所有者权益合计_年初数', '未分配利润_年初数', '应付利息_年初数', '应付票据_年初数', '应付职工薪酬_年初数', '应付账款_年初数', '应交税费_年初数', '预收款项_年初数', '资本公积_年初数', '长期借款_年末数', '负债合计_年末数', '负债和所有者权益总计_年末数', '流动负债合计_年末数', '其他流动负债_年末数', '其他应付款_年末数', '实收资本（股本）_年末数', '所有者权益合计_年末数', '未分配利润_年末数', '应付账款_年末数', '预计负债_年末数', '应交税费_年末数', '预收款项_年末数', '资本公积_年末数', '专项应付款_年末数', '货币资金_年初数', '存货_年初数', '长期股权投资_年初数', '非流动资产合计_年初数', '工程物资_年初数', '固定资产合计_年初数', '固定资产清理_年初数', '固定资产原价_年初数', '库存商品(产成品)_年初数', '可供出售金融资产_年初数', '流动资产合计_年初数', '其他非流动资产_年初数', '其他应收款_年初数', '应收账款_年初数', '应收股利_年初数', '预付款项_年初数', '资产总计_年初数', '在建工程_年初数', '货币资金_年末数', '存货_年末数', '存货中的周转材料_年末数', '长期待摊费用_年末数', '长期应收款_年末数', '非流动资产合计_年末数', '固定资产合计_年末数', '库存商品(产成品)_年末数', '可供出售金融资产_年末数', '流动资产合计_年末数', '其他应收款_年末数', '应收账款_年末数', '一年内到期的非流动资产_年末数', '应收股利_年末数', '资产总计_年末数', '在建工程_年末数', '经营范围2', '负债合计', '流动负债合计', '其他流动负债', '其他应付款', '所有者权益合计', '应付票据', '应付职工薪酬', '应付账款', '预计负债', '应交税费', '一年内到期的非流动负债', '货币资金', '存货', '存货中的原材料', '长期待摊费用', '非流动资产合计', '固定资产原价', '库存商品(产成品)', '开发支出', '流动资产合计', '其他流动资产', '其他应收款', '应收账款', '一年内到期的非流动资产', '应收利息', '应收票据', '预付款项', '资产总计', '企业所得税与增值税之比']

feature_name_use =feature_selected 
fi = []
cv_score = []
test_pred = np.zeros((test.shape[0],))
skf = StratifiedKFold(n_splits=10, random_state=2019, shuffle=True)



for index, (train_index, test_index) in enumerate(skf.split(train_smo, y_smo)):
    print(index)
    train_x, test_x, train_y, test_y = train_smo.iloc[train_index],train_smo.iloc[test_index], y_smo.iloc[train_index], y_smo.iloc[test_index]
    cat_model = CatBoostRegressor(iterations=300,
                              depth=5,
                              learning_rate=0.1, 
                              loss_function='RMSE', 
                              verbose=20, 
#                               early_stopping_rounds=20
                             )
    cat_model.fit(train_x[feature_name_use], train_y, 
             eval_set=(test_x[feature_name_use], test_y))
    y_val = cat_model.predict(test_x[feature_name_use])
    print("roc_auc:", roc_auc_score(test_y, y_val))
    cv_score.append(roc_auc_score(test_y,y_val))
    print("cv_score:", cv_score[index])
    test_pred += cat_model.predict(test[feature_name_use]) / 10
print(cv_score)
test_pred = [0.0 if r < 0  else r for r in test_pred]
test_pred = [1.0 if r > 1.0  else r for r in test_pred]
submission = pd.read_csv('../data/submission.csv')
submission['Label'] = test_pred
submission.to_csv('../result/catboost_1207_2.csv',index=False)

In [ ]:
from matplotlib import pyplot
bold("**catboost特征重要性**")
print(cat_model.feature_importances_)
pyplot.bar(range(len(cat_model.feature_importances_)), cat_model.feature_importances_)
feature_important = dict(zip(feature_name_uesd, cat_model.feature_importances_))
print("catboost特征重要性:", feature_important)
feature_import_usefull = [key for key, value in feature_important.items() if value >0.0]
print("\n\n catboost 重要性 > 0 特征类别：", len(feature_import_usefull),  feature_import_usefull)



In [ ]:
feature_selected = ['企业类型', '登记机关', '企业状态', '投资总额', '注册资本', '行业门类', '企业类别', '管辖机关', '增值税', '企业所得税', '印花税', '教育费', '城建税', '长期借款_年初数', '短期借款_年初数', '非流动负债合计_年初数', '负债合计_年初数', '负债和所有者权益总计_年初数', '流动负债合计_年初数', '其他流动负债_年初数', '其他应付款_年初数', '实收资本（股本）_年初数', '所有者权益合计_年初数', '未分配利润_年初数', '应付利息_年初数', '应付票据_年初数', '应付职工薪酬_年初数', '应付账款_年初数', '应交税费_年初数', '预收款项_年初数', '资本公积_年初数', '长期借款_年末数', '负债合计_年末数', '负债和所有者权益总计_年末数', '流动负债合计_年末数', '其他流动负债_年末数', '其他应付款_年末数', '实收资本（股本）_年末数', '所有者权益合计_年末数', '未分配利润_年末数', '应付账款_年末数', '预计负债_年末数', '应交税费_年末数', '预收款项_年末数', '资本公积_年末数', '专项应付款_年末数', '货币资金_年初数', '存货_年初数', '长期股权投资_年初数', '非流动资产合计_年初数', '工程物资_年初数', '固定资产合计_年初数', '固定资产清理_年初数', '固定资产原价_年初数', '库存商品(产成品)_年初数', '可供出售金融资产_年初数', '流动资产合计_年初数', '其他非流动资产_年初数', '其他应收款_年初数', '应收账款_年初数', '应收股利_年初数', '预付款项_年初数', '资产总计_年初数', '在建工程_年初数', '货币资金_年末数', '存货_年末数', '存货中的周转材料_年末数', '长期待摊费用_年末数', '长期应收款_年末数', '非流动资产合计_年末数', '固定资产合计_年末数', '库存商品(产成品)_年末数', '可供出售金融资产_年末数', '流动资产合计_年末数', '其他应收款_年末数', '应收账款_年末数', '一年内到期的非流动资产_年末数', '应收股利_年末数', '资产总计_年末数', '在建工程_年末数', '经营范围2', '负债合计', '流动负债合计', '其他流动负债', '其他应付款', '所有者权益合计', '应付票据', '应付职工薪酬', '应付账款', '预计负债', '应交税费', '一年内到期的非流动负债', '货币资金', '存货', '存货中的原材料', '长期待摊费用', '非流动资产合计', '固定资产原价', '库存商品(产成品)', '开发支出', '流动资产合计', '其他流动资产', '其他应收款', '应收账款', '一年内到期的非流动资产', '应收利息', '应收票据', '预付款项', '资产总计', '企业所得税与增值税之比']

feature_name_use =feature_selected 
fi = []
cv_score = []
test_pred = np.zeros((test.shape[0],))
skf = StratifiedKFold(n_splits=10, random_state=2019, shuffle=True)



for index, (train_index, test_index) in enumerate(skf.split(train_smo, y_smo)):
    print(index)
    train_x, test_x, train_y, test_y = train_smo.iloc[train_index],train_smo.iloc[test_index], y_smo.iloc[train_index], y_smo.iloc[test_index]
    cat_model = CatBoostRegressor(iterations=300,
                              depth=5,
                              learning_rate=0.1, 
                              loss_function='RMSE', 
                              verbose=20, 
#                               early_stopping_rounds=20
                             )
    cat_model.fit(train_x[feature_name_use], train_y, 
             eval_set=(test_x[feature_name_use], test_y))
    y_val = cat_model.predict(test_x[feature_name_use])
    print("roc_auc:", roc_auc_score(test_y, y_val))
    cv_score.append(roc_auc_score(test_y,y_val))
    print("cv_score:", cv_score[index])
    test_pred += cat_model.predict(test[feature_name_use]) / 10
print(cv_score)
test_pred = [0.0 if r < 0  else r for r in test_pred]
test_pred = [1.0 if r > 1.0  else r for r in test_pred]
submission = pd.read_csv('../data/submission.csv')
submission['Label'] = test_pred
submission.to_csv('../result/catboost_1207_2.csv',index=False)

In [ ]:
print(feature_class1)

In [ ]:

feature_name_use =feature_class1 
fi = []
cv_score = []
test_pred = np.zeros((test.shape[0],))
skf = StratifiedKFold(n_splits=10, random_state=2019, shuffle=True)



for index, (train_index, test_index) in enumerate(skf.split(train_smo, y_smo)):
    print(index)
    train_x, test_x, train_y, test_y = train_smo.iloc[train_index],train_smo.iloc[test_index], y_smo.iloc[train_index], y_smo.iloc[test_index]
    cat_model = CatBoostRegressor(iterations=300,
                              depth=5,
                              learning_rate=0.1, 
                              loss_function='RMSE', 
                              verbose=20, 
#                               early_stopping_rounds=20
                             )
    cat_model.fit(train_x[feature_name_use], train_y, 
             eval_set=(test_x[feature_name_use], test_y))
    y_val = cat_model.predict(test_x[feature_name_use])
    print("roc_auc:", roc_auc_score(test_y, y_val))
    cv_score.append(roc_auc_score(test_y,y_val))
    print("cv_score:", cv_score[index])
    test_pred += cat_model.predict(test[feature_name_use]) / 10
print(cv_score)
test_pred = [0.0 if r < 0  else r for r in test_pred]
test_pred = [1.0 if r > 1.0  else r for r in test_pred]
submission = pd.read_csv('../data/submission.csv')
submission['Label'] = test_pred
submission.to_csv('../result/danmoxing_smote.csv',index=False)